In [135]:
import pandas as pd
import numpy as np

In [136]:
# // get geography files
warnCounties = pd.read_excel('data/warnLogs_200720.xlsx',
                             sheet_name="warnCounties",
                             converters={'FIPS':str})

warnCities= pd.read_excel('data/warnLogs_200720.xlsx',
                          sheet_name="warnCities",
                          usecols="A,B,E,F")

msa = pd.read_excel('../data/geographies/cbsa.xlsx',
                    sheet_name="cbsa",
                    usecols="A,D,M",
                    converters={'FIPS_COUNTY':str,
                                'CBSA Code':str})

msa.rename(columns={'CBSA Code':'CBSA','CBSA Title':'MSA','FIPS_COUNTY':'FIPS'}, inplace=True)

zipWalk = pd.read_excel('../data/spatial/zips/Zip_to_zcta_crosswalk_2020.xlsx',
                        sheet_name='ziptozcta2019',
                        usecols='A,E',
                        converters={'ZIP_CODE':str,
                                    'ZCTA':str})

In [137]:
data = pd.read_excel('data/warnLogs_200720.xlsx',
                     sheet_name="warnLogs",
                     usecols="A:F,H",
                     converters={'ZIP':str})

data['Companies'] = 1

# trim all columns
data['Company name'] = data['Company name'].str.strip()
data['City'] = data['City'].str.strip()
data['ZIP'] = data['ZIP'].str.strip()
data['County'] = data['County'].str.strip()

data.rename(columns={'County':'warnCounty',
                     'City':'warnCity',
                     'Company name':'Company',
                     'Est. Impact':'Employees'}, inplace=True)

In [138]:
data = pd.merge(data, warnCounties, how='left', on='warnCounty')
data = pd.merge(data, warnCities, how='left', on='warnCity')
data = pd.merge(data, msa, how='left', on='FIPS')

In [139]:
data = pd.merge(data, zipWalk, how='left', left_on='ZIP', right_on='ZIP_CODE')

In [140]:
data = data[['Date','Company','Companies','Employees','City',
             'ZIP','County','FIPS','MSA','CBSA','ZCTA']]

In [141]:
data.sort_values('Date', inplace=True)

In [142]:
data.reset_index(drop=True, inplace=True)

In [143]:
data['Month'] = data['Date'].dt.strftime('%b')
data['Year'] = data['Date'].dt.strftime('%Y')

In [144]:
data = data[['Date','Month','Year',
             'Company','City','ZIP','County','FIPS','MSA','CBSA','ZCTA',
             'Companies','Employees']]

In [145]:
data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d').dt.strftime('%m-%d-%Y')

In [146]:
data.head()

,Date,Month,Year,Company,City,ZIP,County,FIPS,MSA,CBSA,ZCTA,Companies,Employees
0,01-04-2007,Jan,2007,"Polymer Group, Inc",Gainesville,30504,Hall County,13139,"Gainesville, GA",23580,30504,1,58
1,01-08-2007,Jan,2007,Enterprise Fund Distributors,Atlanta,30326,Fulton County,13121,"Atlanta-Sandy Springs-Alpharetta, GA",12060,30326,1,89
2,01-08-2007,Jan,2007,Marriott Atlanta Global Reservations Sales & C...,Sandy Springs,30328,Fulton County,13121,"Atlanta-Sandy Springs-Alpharetta, GA",12060,30328,1,153
3,01-08-2007,Jan,2007,"Glen Raven Custom Fabrics,llc",Elberton,30635,Elbert County,13105,NaN,NaN,30635,1,54
4,01-08-2007,Jan,2007,Schwan's Bakery Inc.,Suwanee,30024,Gwinnett County,13135,"Atlanta-Sandy Springs-Alpharetta, GA",12060,30024,1,200


In [147]:
# write files
data.to_csv('application/data/warnLogs.csv', index=False)
# zctaRoll.to_csv('application/data/zctaRoll.csv', index=False)
# cityRoll.to_csv('application/data/cityRoll.csv', index=False)

In [148]:
dataJson = data.copy()

In [149]:
dataJson.head()

,Date,Month,Year,Company,City,ZIP,County,FIPS,MSA,CBSA,ZCTA,Companies,Employees
0,01-04-2007,Jan,2007,"Polymer Group, Inc",Gainesville,30504,Hall County,13139,"Gainesville, GA",23580,30504,1,58
1,01-08-2007,Jan,2007,Enterprise Fund Distributors,Atlanta,30326,Fulton County,13121,"Atlanta-Sandy Springs-Alpharetta, GA",12060,30326,1,89
2,01-08-2007,Jan,2007,Marriott Atlanta Global Reservations Sales & C...,Sandy Springs,30328,Fulton County,13121,"Atlanta-Sandy Springs-Alpharetta, GA",12060,30328,1,153
3,01-08-2007,Jan,2007,"Glen Raven Custom Fabrics,llc",Elberton,30635,Elbert County,13105,NaN,NaN,30635,1,54
4,01-08-2007,Jan,2007,Schwan's Bakery Inc.,Suwanee,30024,Gwinnett County,13135,"Atlanta-Sandy Springs-Alpharetta, GA",12060,30024,1,200


In [150]:
dataJson.to_json('application/data/warnLogs.json', orient='index')

In [ ]:
# get month and year columns

# rollup by zcta, month, and year with ttal companies and total employees
# rollup by city, month, and year with total companies and total employees
# write all files as csv and publish to github repo